In [ ]:
import os
import random
import networkx as nx

In [ ]:
def make_dir(path):
    try:
        os.mkdir(path)
    except OSError as error:
        return True
    
def check_dir():
    for homedir in ["./wcnf/", "./wcnf+/"]:
        make_dir(homedir)
            
check_dir()

## Generate planted partition graph

In [ ]:
def gen_planted_graph(n, m, seed):
    random.seed(seed)
    file_cnf = "wcnf/" + str(seed).zfill(3) + ".wcnf"
    file_xor = "wcnf+/" + str(seed).zfill(3) + ".wcnfp"
    G = nx.planted_partition_graph(n, m, 0.5, 0.5, seed)
    weight = 0
    for (u, v) in G.edges():
        if G.nodes[u]["block"] == G.nodes[v]["block"]:
            G[u][v]["weight"] = 1
            weight += 1
        else:
            G[u][v]["weight"] = 2
            weight += 2
            
    f_cnf = open(file_cnf, 'w')
    f_xor = open(file_xor, 'w')
    f_cnf.write('p wcnf {} {} {}\n'.format(n*m, 2*len(G.edges()), 1+weight*2))
    f_xor.write('p wcnf+ {} {} {}\n'.format(n*m, len(G.edges()), 1+weight))
    for (u, v) in G.edges():
        f_cnf.writelines(  '{} {} {} 0\n'.format(G[u][v]["weight"],  u+1,  v+1))
        f_cnf.writelines(  '{} {} {} 0\n'.format(G[u][v]["weight"], -u-1, -v-1))
        f_xor.writelines('x {} {} {} 0\n'.format(G[u][v]["weight"],  u+1,  v+1))
    f_cnf.close()
    f_xor.close()

In [ ]:
for i in range(400):
    if i < 100:
        gen_planted_graph(32, 32, i+1)
    elif i < 200:
        gen_planted_graph(16, 32, i+1)
    elif i < 300:
        gen_planted_graph(32, 16, i+1)
    else:
        gen_planted_graph(16, 16, i+1)

## Convert WCNF from MSE2019

In [ ]:
dir_path = 'mse2019/'
files = [(f, os.path.getsize(os.path.join(dir_path, f))) for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
files = sorted(files, key=lambda x: x[1])

In [ ]:
for i in range(len(files)):
    f_src = open(dir_path+files[i][0], "r")
    f_cnf = open("wcnf/"  + str(i+401) + ".wcnf",  "w")
    f_xor = open("wcnf+/" + str(i+401) + ".wcnfp", "w")
    for line in f_src:
        split = line.split()
        if line[0] == 'c':
            f_cnf.writelines(line)
            f_xor.writelines(line)
            continue
        if line[0] == 'p':    # Read p line
            n, m, o = split[2:]
            f_cnf.writelines('p wcnf {} {} {}\n'.format(n, m, o))
            f_xor.writelines('p wcnf+ {} {} {}\n'.format(n, int(int(m)/2), int((int(o)-1)/2+1)))
        else:
            w, x, y, _ = split
            f_cnf.writelines(line)
            if (float(x) > 0):
                f_xor.writelines('x {}'.format(line))
    f_src.close()
    f_cnf.close()
    f_xor.close()